In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d mohitsingh1804/plantvillage

In [ ]:
!unzip plantvillage.zip -d /content/PlantVillage

In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
import sys
import io

# Fix output encoding
# sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

# Step 1: Data Preparation
# Using ImageDataGenerator for data augmentation and rescaling
train_data_gen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    rotation_range=20,        # Random rotation
    width_shift_range=0.2,    # Random horizontal shifts
    height_shift_range=0.2,   # Random vertical shifts
    shear_range=0.2,          # Random shear transformations
    zoom_range=0.2,           # Random zoom transformations
    horizontal_flip=True,     # Randomly flip images horizontally
    fill_mode='nearest',      # Fill in new pixels
    validation_split=0.2      # Use 20% of the data for validation
)

# Load training data from directory, with training and validation split
train_generator = train_data_gen.flow_from_directory(
    '/content/PlantVillage/PlantVillage/train',  # The root directory containing all class subfolders
    target_size=(224, 224),   #	The images are resized to 224x224 pixels
    batch_size=32,
    class_mode='categorical',   # indicates multi-class classification.
    subset='training'
)

validation_generator = train_data_gen.flow_from_directory(
    '/content/PlantVillage/PlantVillage/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Step 2: Model Building
# Use VGG16 pre-trained on ImageNet as a base model for transfer learning
from tensorflow.keras.layers import Input

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the layers

# Add custom layers for the new classification task
model = Sequential([
    Input(shape=(224, 224, 3)),  # Explicit input layer
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model's architecture
base_model.summary()  # Check the base model
model.summary()       # Check the complete model


# Step 3: Train the Model
# Add learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Training the model with the training and validation data
history = model.fit(
    train_generator,
    epochs=25,                        # Increased number of training epochs
    validation_data=validation_generator,
    callbacks=[reduce_lr]
)

# Step 4: Evaluate the Model
# Evaluate the trained model on the validation data
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Step 5: Plot Training History
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.show()

# Step 6: Save the Model
# Save the trained model for future use
model.save('leaf_disease_model.h5')